In [ ]:
#confirm two main goals of lab: 1) walk students through the modelling generalization, and 
#2) get them excited about doing it on real data. 

#

# Cornell Hockey Lab - Extention of Baseball Elimination

**Objectives:**
* edit this
* edit this

**Key Ideas:**
* integrality property edit this!!!!!!!
* max-flow min-cut theorem
* the baseball elimination problem

**Reading Assignment:**
* edit this

**Brief description:** edit this

In [ ]:
# imports
from selenium import webdriver
from bs4 import BeautifulSoup
import pandas as pd
from datetime import datetime, timedelta
import re
import numpy as np
import networkx as nx

## Part 1: Hockey Elimination - Toy Exmaple

You have seen the Baseball Elimination process, and learned how the max flow problem can determine whether a team is eliminated. Unlike baseball, Hockey has ties. As a result, we have to count points instead of just wins and losses. For each game,
* if the result IS NOT tied, the winning team receives 2 points, and the losing team receives 0 points;
* if the result IS tied, both teams receive 1 point.

Before we can explore Cornell's epic season (both men and women!) in Ice Hockey, here is the data from a toy season for only 4 teams. The 4 teams are numbered 1, 2, 3, and 4. And here are results from the first 8 games.

Teams in Games Played | Result 
--- | ---
1,2 | 1 won
1,3 | tied
2,3 | 2 won
3,4 | 3 won
1,2 | 1 won
1,3 | 1 won
2,3 | tied
3,4 | 3 won


We can calculate points for each team based on game results. `Team 1` has won 3 games and tied one game, so it has 2*3 + 1 = 7 points.

**Q1:** Please calculate the points for `team 2` based on game results to make sure you understand the scoring system.

**A:**

<font color='blue'> Team 2 score: 2+1 = 3</font>

Below is the current point standing for all the teams after the first 8 games.

Team | Points
--- | ---
1 | 7
2 | 3
3 | 6
4 | 0

And the table below shows Games remaining to be played:

vs. | 1 | 2 | 3 | 4
 -- | -- | -- | -- | --
**1** | - | 1 | 1 | 2
**2** | 1 | - | 2 | 2
**3** | 1 | 2 | - | 1
**4** | 2 | 2 | 1 | -

Team 4, the team we are rooting for, has not won any points so far. We would like to determine whether it still has a chance to finish first at the end of the season (we are satisfied with a tie for first place as well).

**Q2:** How many points can team 4 possibly win during the season? Call this number $P$. (Hint: assume that team 4 wins all their remaning games.)

In [ ]:
# TODO: uncomment and set P to the appropriate value
# P = 

### BEGIN SOLUTION
P = 10
### END SOLUTION

Also, we define data for the Hockey Elimination problem as the following:

* a collection of teams $1, 2, \ldots, n$,
* the number of games $g(i,j)$ remaining to be played between teams $i$ and $j$ for all pairs of teams $i$ and $j$,
* the number of points $p(i)$ team $i$ already has.

Team 1 has already won 7 points, or we can say $p(i) = 7$.  

**Q3:** At most how many more points is this team allowed to win if we want to make sure that our team comes out first? What about teams 2 and 3?

**A:**  <font color='blue'> An upper bound on the number of points team 1 could win is 10 - 7 = 3. Team 2 could win at most 7 more points. Team 3 could win at most 4 more points.</font>

**Q4:** How can you express this amount in general (in terms of $P$ and $p(i)$) for team $i$?

**A:** <font color='blue'> $P - p(i)$</font>

Just like Baseball Elimination before, we are going to create the max flow instance. This description below of max flow is very similar to that of Baseball Elimination, except for `games` become `points`.

If one of two teams scheduled to play is team 4, then we assume that the game outcome is decided in our favor (team 4 wins and receives 2 points). For the rest of the games, we would like to assign a winner so that no team wins more points than the number determined above. So for each pair of teams (other than those containing our team) we would like to decide how many of the leftover games between them are going to be won by one team or the other, or be tied.

To illustrate this, we draw two sets of nodes: one set for all the pairs of teams not containing our team (these will be called the *pair nodes*) and another one for the individual teams (these are called the *team nodes*). We can interpret $g(i,j)$, the number of games remaining to be played between teams $i$ and $j$, as the amount of "excess games" at the pair node. And each excess game has 2 points that can be distributed. These points need to be "distributed" between the two corresponding team nodes as represented by edges.  

The graph below is generated for you since you have already seen it in Baseball Elimination.

In [ ]:
G = nx.DiGraph()
# pair nodes
G.add_node('1,2', pos=(10,30))
G.add_node('1,3', pos=(10,20))
G.add_node('2,3', pos=(10,10))
# team nodes
G.add_node('1', pos=(20,30))
G.add_node('2', pos=(20,20))
G.add_node('3', pos=(20,10))

# Remember that they are directed edges, so the order of endpoints matter
G.add_edge('1,2', '1')
G.add_edge('1,2', '2')
G.add_edge('1,3', '1')
G.add_edge('1,3', '3')
G.add_edge('2,3', '2')
G.add_edge('2,3', '3')

# graph display
pos=nx.get_node_attributes(G,'pos')
nx.draw_networkx(G,pos,node_size=1000,node_color='lightblue')

To make a single-source, single-sink flow problem from this model, we introduce two nodes: a node which is the "source
of all games" and a node which is the "sink of all played games".  

**Q5:** What should be the capacity of (source, pair node) arcs? What about (team node, sink) arcs? Remember we are sending points instead of wins through the edges!

**A:** <font color='blue'> The capacity of (source, pair node) arcs should be 2*g(i,j). The capacity of (team node, sink) arcs should be $P - p(i).$</font>

**Q6:** Fill in the capacity values missing in the code below and run the cell to display the graph.

In [ ]:
# adding our source node and sink node
G.add_node('s', pos=(0,20))
G.add_node('t', pos=(30,20))

# TODO: Uncomment and fill in the capacities
G.add_edge('s', '1,2', capacity = 2)
G.add_edge('s', '1,3', capacity = 2)
# G.add_edge('s', '2,3', capacity = X)
# G.add_edge('1', 't', capacity = X)
# G.add_edge('2', 't', capacity = X)
# G.add_edge('3', 't', capacity = X)

### BEGIN SOLUTION
G.add_edge('s', '2,3', capacity = 4)
G.add_edge('1', 't', capacity = P-7)
G.add_edge('2', 't', capacity = P-3)
G.add_edge('3', 't', capacity = P-6)
### END SOLUTION

# graph display
pos=nx.get_node_attributes(G,'pos')
cap=nx.get_edge_attributes(G,'capacity')
nx.draw_networkx(G,pos,node_size=1000,node_color='lightblue')
nx.draw_networkx_edge_labels(G,pos,edge_labels=cap);

**Q7:** What does a feasible flow, with all integer flow values, in the above network correspond to? How can we interpret the value of a flow? 

**A:** <font color='blue'>We can interpret the flow on $(s,(i,j))$ to be the number of new points between teams $i$ and $j$, $((i,j), i)$ arcs to be the number of points team $i$ won aganist team $j$, and $(i,t)$ arcs to be the total number of addtional points team $i$ has. </font>

**Q8:** What does the value of the max flow have to equal to for a team to not be eliminated?

**A:** <font color='blue'>The value of the max flow has to equal the number of remaining points for all teams $(i,j)$ where $i \neq 4$, $j \neq 4$. </font>

In [ ]:
# The max flow problem is solved in this cell.
flow_value, flow = nx.maximum_flow(G, 's', 't')
print("The value of the flow is", flow_value)
for i, j in G.edges:
    print("The flow on the arc from "+i+" to "+j+" is",flow[i][j])

If the max flow is equal to the number of remaining points for all teams $(i,j)$ where $i \neq 4$, $j \neq 4$, all remaining points are distributed and it's possible that no team gets more points than team 4. We know the value of the max flow is 8 from the cell above.

**Q9:** What is the minimum cut (i.e., which nodes are on the set $S$)? Has our team been eliminated already or not yet? If team 4 WAS NOT eliminated, then give the result of the remaining games among the three other teams based on the max flow calculated by the computer program above. Explain everything like you would to someone who knows nothing about max flow or min cut problems.

**A:** <font color='blue'>The minimum cut is $S = {s}$. The team has not been eliminated yet. It is possible to have all the points from remaining games amount teams 1, 2, and 3 distributed given that 4 must win. Team 1 and 2 tie the game between 1,2. Team 1 wins the game between 1,3. Team 2 wins the two games between 2,3.  </font>  

## Part 2: Proof of Elimination

The standing (number of points each team has) is the same. The games remaining to be played change.

Team | Points
--- | ---
1 | 7
2 | 3
3 | 6
4 | 0

Games remaining to be played:

vs. | 1 | 2 | 3 | 4
 -- | -- | -- | -- | --
**1** | - | 3 | 3 | 2
**2** | 3 | - | 2 | 2
**3** | 3 | 2 | - | 1
**4** | 2 | 2 | 1 | -

**Q10:** Finish the code below and display the graph. We are once again cheering for team 4.

In [ ]:
# creating a new graph
G = nx.DiGraph()
G.add_node('s', pos=(0,20))
G.add_node('t', pos=(30,20))
G.add_node('1,2', pos=(10,30))
G.add_node('1,3', pos=(10,20))
G.add_node('2,3', pos=(10,10))
G.add_node('1', pos=(20,30))
G.add_node('2', pos=(20,20))
G.add_node('3', pos=(20,10))

# TODO: Uncomment and fill in the remaining edges
# Remember that they are directed edges, so the order of endpoints matter
G.add_edge('1,2', '1')
G.add_edge('1,2', '2')
G.add_edge('1,3', '1')
G.add_edge('1,3', '3')
# G.add_edge('XXX', 'X')
# G.add_edge('XXX', 'X')

### BEGIN SOLUTION
G.add_edge('2,3', '2')
G.add_edge('2,3', '3')
### END SOLUTION

# TODO: Uncomment and fill in the capacities
# G.add_edge('s', '1,2', capacity = X)
# G.add_edge('s', '1,3', capacity = X)
# G.add_edge('s', '2,3', capacity = X)
# G.add_edge('1', 't', capacity = X)
# G.add_edge('2', 't', capacity = X)
# G.add_edge('3', 't', capacity = X)

### BEGIN SOLUTION
G.add_edge('s', '1,2', capacity = 6)
G.add_edge('s', '1,3', capacity = 6)
G.add_edge('s', '2,3', capacity = 4)
G.add_edge('1', 't', capacity = P-7)
G.add_edge('2', 't', capacity = P-3)
G.add_edge('3', 't', capacity = P-6)
### END SOLUTION

pos=nx.get_node_attributes(G,'pos')
cap=nx.get_edge_attributes(G,'capacity')
nx.draw_networkx(G,pos,node_size=1000,node_color='lightblue')
nx.draw_networkx_edge_labels(G,pos,edge_labels=cap);

**Q11:** Can you tell if team 4 is eliminated without running max flow? Explain why to your friend who doesn't know anything about Operations Research.

**A:** <font color='blue'> Team 4 is eliminated. There are a total of 6+6+4=16 points that need to be distributed among team 1, 2, and 3 given 4 must win. However, only 3+7+4=14 points can be won by all 3 teams without causing team 4 to lose. </font>  

The previous question can be answered without running max flow algorithm because it has a trivial cut, where the number of points that need to be distributed is larger than the total number of points all 3 other teams can win. And let's look at another scenario where the games left to be played is once again different, while the standing stays the same.

The standing:

Team | Points
--- | ---
1 | 7
2 | 3
3 | 6
4 | 0

Games remaining to be played:

vs. | 1 | 2 | 3 | 4
 -- | -- | -- | -- | --
**1** | - | 1 | 4 | 2
**2** | 1 | - | 1 | 2
**3** | 4 | 1 | - | 1
**4** | 2 | 2 | 1 | -

The code to define the graph (nodes, edges, capacities) are written for you below.

In [ ]:
# creating a new graph
G = nx.DiGraph()
G.add_node('s', pos=(0,20))
G.add_node('t', pos=(30,20))
G.add_node('1,2', pos=(10,30))
G.add_node('1,3', pos=(10,20))
G.add_node('2,3', pos=(10,10))
G.add_node('1', pos=(20,30))
G.add_node('2', pos=(20,20))
G.add_node('3', pos=(20,10))

# Edges
G.add_edge('1,2', '1')
G.add_edge('1,2', '2')
G.add_edge('1,3', '1')
G.add_edge('1,3', '3')
G.add_edge('2,3', '2')
G.add_edge('2,3', '3')

# Capacities
G.add_edge('s', '1,2', capacity = 2)
G.add_edge('s', '1,3', capacity = 8)
G.add_edge('s', '2,3', capacity = 2)
G.add_edge('1', 't', capacity = P-7)
G.add_edge('2', 't', capacity = P-3)
G.add_edge('3', 't', capacity = P-6)

pos=nx.get_node_attributes(G,'pos')
cap=nx.get_edge_attributes(G,'capacity')
nx.draw_networkx(G,pos,node_size=1000,node_color='lightblue')
nx.draw_networkx_edge_labels(G,pos,edge_labels=cap);

In [ ]:
# Here is the max flow result given by the computer:
flow_value, flow = nx.maximum_flow(G, 's', 't')
print("The value of the flow is", flow_value)
for i, j in G.edges:
    print("The flow on the arc from "+i+" to "+j+" is",flow[i][j])

**Q12:** Based on the max flow result, has our team been eliminated? Justify it for a friend that doesn't understnad max flow. What is the minimum cut (i.e., which nodes are on the set $S$)?

**A:** <font color='blue'> It has been eliminated. The value of the max flow is less than the total number of remaining points. As discussed in `Q8`, the we can't push all the points needed to be played among the other teams given team 4 wins to the sink. Nodes in set $S$ are `s`, `1,3`, `1`, `3`. </font>  

**Q13:** In the previous scenario, we concluded that team 4 is eliminated without running max flow because it has a trivial cut. Here, can you say that team 4 is eliminated without running max flow?

**A:** <font color='blue'> We can't say team 4 is eliminated. The number of points in remaining games (12 points) is no more than the total points the three other teams are allowed to win (14 points).</font> 

**Q14:** Let's interpret the min cut. What does the inclusion of a pair nodes in $S$ say about the inclusion of the corresponding pair of team nodes? Also, explain what including a pair node in the min cut means.

**A:** <font color='blue'> If a pair node is included in the min cut, both of the teams most be included in the min cut since the edge capacity between these are infinity. When a pair node is included in the min cut, at least one of the teams will end up receiving more points than our team in interest in the end.</font>  

**In conclusion:**

* If `max flow value` = the sum over all pairs of teams (i,j) of 2*g(i,j), for all teams $(i,j)$ where $i,j \neq $ the team we are interested in and and i<j, team i IS NOT eliminated.
* If `max flow value` < the sum over all pairs of teams (i,j) of 2*g(i,j), for all teams $(i,j)$ where $i,j \neq $ the team we are interested in and and i<j, team i IS eliminated.

**Q15:** Why does the max flow still work for points in hockey instead of wins in baseball? Let's verify this for both of the bullet points above.

**A:** <font color='blue'> 1st bullet point: if max flow = the sum over all pairs of teams (i,j) of 2*g(i,j), that means there is a feasible way to assign all the points in the remaining games to all teams but the one we are interested in with our team of interest winning. 2nd bullet point: we can't do what we stated in the first bullet point.</font>  

## Part 3: First Look at 2019-20 ECAC Hockey

Everyone knows that Cornell has an amazing ice hockey program. In the 2019-20 season, both the men's and women's teams won the ECAC championship (The Ivies minus UPenn and Columbia, plus some other schools in the northeast). We have retrieved the data from the website and our code imports the csv file of the men's season's results into a table. Each row of the dataframe contains a game and its result.

`Note:` some of the code in this section is beyond the scope of this course. You do not need to understand all the code here, but instead should focus on how the max flow problem works here.

In [ ]:
#the list of teams in ECAC
teams = ['Brown','Clarkson','Colgate','Cornell','Dartmouth','Harvard','Princeton','Quinnipiac',
         'Rensselaer','St. Lawrence','Union','Yale']

In [ ]:
results = pd.read_csv('data/ECAC_19-20_men.csv')
results['date'] = pd.to_datetime(results['date'], format='%m/%d/%y')
# To view the results table, just type: results
results

In [ ]:
id_to_teams = teams 
teams_to_id = {id_to_teams[i] : i for i in range(len(teams))}

We also give you the function called `instance` that outputs how many points each team has and how many games are left at any given date. The function counts the wins from the `start` date to `now`, and the number of remaning games from `now` to the `end`. You can only input one datetime, `now`, and the default start and end are the start and end of the season. Or you can put in custom start, now, and end dates.

`instance` outputs the wins and games left as a vector.

In [ ]:
def instance(*args):
    if len(args) == 1:
        start = datetime(2019, 10, 25)
        now = args[0]
        end = datetime(2020, 2, 29)
    elif len(args) == 3:
        start = args[0]
        now = args[1]
        end = args[2]
    w = {}
    past = results[(results.date >= start) & (results.date <= now)]
    future = results[(results.date > now) & (results.date <= end)]
    for team in teams:
        w[team] = 2*len(past[past.winner == team])
    for index, row in past[past.winner == 'tie'].iterrows():
        w[row['team_1']] = w[row['team_1']] + 1
        w[row['team_2']] = w[row['team_2']] + 1
    w = pd.DataFrame(w, index=[0]).transpose().rename(columns={0 : 'w'})
    
    games_left = np.zeros((len(teams), len(teams)))
    for i in teams:
        for j in teams:
            rem = len((future[((future.team_1 == i) & (future.team_2 == j)) |
                              ((future.team_1 == j) & (future.team_2 == i))]))
            games_left[teams_to_id[i], teams_to_id[j]] = rem
            games_left[teams_to_id[j], teams_to_id[i]] = rem
    games_left = pd.DataFrame(index=teams, columns=teams, data=games_left).astype(int)
            
    return (w, games_left)

The season starts on 10/25/2019, and ends on 02/29/2020. We are interested in the data from the entire season so these two dates are our start and end dates. We are going to look at the standings and games left on 11/02/2019, only 5 games have been played since the start of season. The games that were played are displayed when we first imported the csv file, and is also displayed below. Also, only one game, the one between Yale and Brown was played on 11/02/2019.


index | date | team_1 | score_1 | team_2 | score_2 | winner
-- | -- | -- | -- | -- | -- | --
**0** | 2019-10-25 | Rensselaer | 3 | Union | 2 | Rensselaer
**1** | 2019-10-26 | Union | 2 | Rensselaer | 1 | Union
**2** | 2019-11-01 | Brown | 2 | Yale | 3 | Yale
**3** | 2019-11-01 | Dartmouth | 3 | Harvard | 7 | Harvard
**4** | 2019-11-02 | Yale | 1 | Brown | 5 | Brown

In [ ]:
# the line below is how you use the function 'instance'
w, games_left = instance(datetime(2019, 10, 25), datetime(2019, 11, 2), datetime(2020, 2, 29))
# we reset the index in this case to make our code in the next sections easier
w = w.reset_index()
#display the dataframe by just typing what we want to show: 'w'
w

We can very easily check that `w` above can be obtained from calculating points using games played. We can also display games_left, we are not going to verify the calculation here.

In [ ]:
games_left

We are going to walk through the code that determines whether a team is eliminated (again, no need to understand the code). Obviously, our team of interest or the team we are rooting for is Cornell. And the date of interest is still 11/02/2019, with the start and end date being the start and end of the season.

In [ ]:
#we want the id of the team given the name. We use .index() to search for the name in our list teams.
interested_team = "Cornell"
team_id = teams.index(interested_team)
#we first see how many points Cornell has won so far by 11/03/2019
print('Cornell has won this many points so far: ', w.iloc[team_id,1])
#we then find the sum of all g(Cornell, j), the total number of games left for Cornell to play
potential_games = 0
for i in range(len(teams)):
    potential_games += games_left.iloc[team_id,i]
#the total number of points possible is 2*potential number of games + points so far
tot_pts = 2*potential_games + w.iloc[team_id,1]

print('Cornell can at best win this many points this season: ', tot_pts)

In [ ]:
#we are making the max flow instance for the rest of the 11 teams, and we call that list of teams 'teams_11'
teams_11 = teams.copy()
teams_11.remove(interested_team)

We are going to create the max flow graph. Notice that we don't display the graph here because the number of nodes make it impossible to visualize. We don't have enough space on the screen and it takes very long.

In [ ]:
G = nx.DiGraph()

# pair nodes
node_name_list = []
for i in range(len(teams_11)):
    for j in range(len(teams_11)):
        if i<j:
            node_name = str(teams_11[i]+', '+teams_11[j])
            node_name_list.append(node_name)
            G.add_node(node_name, pos=(10, len(teams_11)-i))

# team nodes
for i in range(len(teams_11)):
    G.add_node(teams_11[i], pos=(20, len(teams_11)-i))

# add edges between pair nodes and team nodes
for i in range(len(teams_11)):
    for j in range(len(teams_11)):
            if i<j:
                node_name = str(teams_11[i]+', '+teams_11[j])
                G.add_edge(node_name, teams_11[i])
                G.add_edge(node_name, teams_11[j])

# adding our source node and sink node
G.add_node('s', pos=(0,20))
G.add_node('t', pos=(30,20))

# fills in capacities
tot_left_others = 0
for i in range(len(teams_11)):
    for j in range(len(teams_11)):
            if i<j:
                node_name = str(teams_11[i]+', '+teams_11[j])
                G.add_edge('s', node_name, capacity=2*games_left.iloc[i,j])
                tot_left_others += 2*games_left.iloc[i,j]
sum_allowed = 0
for i in range(len(teams_11)):
    G.add_edge(teams_11[i], 't', capacity=tot_pts-w.iloc[i,1])
    sum_allowed += (tot_pts-w.iloc[i,1])

Based on the conclusion in part 2, 
* If `max flow value` = the sum over all pairs of teams (i,j) of 2*g(i,j), for all teams $(i,j)$ where $i,j \neq $ the team we are interested in and and i<j, team i IS NOT eliminated.
* If `max flow value` < the sum over all pairs of teams (i,j) of 2*g(i,j), for all teams $(i,j)$ where $i,j \neq $ the team we are interested in and and i<j, team i IS eliminated.

From the cell above, we have already stored $\sum_{i=1}^{n} 2*g(i,j)$ in `tot_left_others`, and also stored the sum of all $P - p(i)$ in `sum_allowed`.

**Q16:** Complete the code in the next cell, by typing in tot_left_others and sum_allowed in the appropriate question mark.

In [ ]:
# TODO: uncomment and fill in the variable name that would go in the question marks
#print('Total sum of points left among all teams but the team in interest: ', ?)
#print('Total points allowed if team in interest must win: ', ?)

### BEGIN SOLUTION
print('Total sum of points left among all teams but the team in interest: ', tot_left_others)
print('Total points allowed if team in interest must win: ', sum_allowed)
### END SOLUTION

Let's solve for the max flow using the computer since we have already set up the graph:

In [ ]:
flow_value, flow = nx.maximum_flow(G, 's', 't')
print("The value of the flow is", flow_value)

**Q17:** Based on the max flow, is Cornell eliminated? Why or why not?

**A:** <font color='blue'>Cornell is not eliminated, because the value of the max flow = sum of points left among all teams but the team in interest.</font>  

## Part 4: General Hockey Elimination in ECAC

Now that we know how to look at whether a specific team has been eliminated at a specific date, we can write a general function to check whether a team is eliminated. Please fill in the cell below based on the code in Part 3.

**Q18:** Fill in the condition at the end of the next code block. It should be one of these two options: "flow_value>tot_left_others" and "flow_value<tot_left_others"

In [ ]:
def if_eliminated(team, date):
    w, games_left = instance(datetime(2019, 10, 25), date, datetime(2020, 2, 29))
    w = w.reset_index()
    team_id = teams.index(team)
    #team_id = 11
    #figure out the P
    potential_games = 0
    for i in range(len(teams)):
        potential_games += games_left.iloc[team_id,i]
    tot_pts = 2*potential_games + w.iloc[team_id,1]
    teams_11 = teams.copy()
    teams_11.remove(team)

    G = nx.DiGraph()
    #pair nodes
    node_name_list = []
    for i in range(len(teams_11)):
        for j in range(len(teams_11)):
            if i<j:
                node_name = str(teams_11[i]+', '+teams_11[j])
                node_name_list.append(node_name)
                G.add_node(node_name, pos=(10, len(teams_11)-i))
    #team nodes
    for i in range(len(teams_11)):
        G.add_node(teams_11[i], pos=(20, len(teams_11)-i))
    #add edges between pair nodes and team nodes
    for i in range(len(teams_11)):
        for j in range(len(teams_11)):
                if i<j:
                    node_name = str(teams_11[i]+', '+teams_11[j])
                    G.add_edge(node_name, teams_11[i])
                    G.add_edge(node_name, teams_11[j])
    #adding our source node and sink node
    G.add_node('s', pos=(0,20))
    G.add_node('t', pos=(30,20))
    #fill in capacities
    tot_left_others = 0
    for i in range(len(teams_11)):
        for j in range(len(teams_11)):
                if i<j:
                    node_name = str(teams_11[i]+', '+teams_11[j])
                    G.add_edge('s', node_name, capacity=2*games_left.iloc[i,j])
                    tot_left_others += 2*games_left.iloc[i,j]
    sum_allowed = 0
    for i in range(len(teams_11)):
        G.add_edge(teams_11[i], 't', capacity=tot_pts-w.iloc[i,1])
        sum_allowed += (tot_pts-w.iloc[i,1])
    
    #if sum of capacities of (s, pair nodes) > sum of (teams, t), the team gets eliminated
    if tot_left_others>sum_allowed:
        return True
    #use max flow to check whether a team is eliminated
    flow_value, flow = nx.maximum_flow(G, 's', 't')
    
    # TODO: uncomment and fill in the condition for the if statement below
    #if ?<?:
        #return True
    
    ### BEGIN SOLUTION
    if flow_value<tot_left_others:
        return True
    ### END SOLUTION
    
    return False

Now, if you want to do what we did in Part 3, you just type the line below:

In [ ]:
if_eliminated('Cornell', datetime(2019, 11, 2))

It checks out with part 3, Cornell was not eliminated on 11/02/2019.

**Q19:** Please write code to check whether `St.Lawrence` was elimated on 01/01/2020 and 02/01/2020.

**A:** <font color='blue'>St. Lawrence was eliminated on 02/01/2020 but not on 01/01/2020</font>  

In [ ]:
# write code here

### BEGIN SOLUTION
if_eliminated('St. Lawrence', datetime(2020, 1, 1))
if_eliminated('St. Lawrence', datetime(2020, 2, 1))
### END SOLUTION

## Part 4: Elimination vs. Winning

The code below displays the order in which teams are eliminated in the 2019-2020 men's ECAC season. Note that more than one team can be eliminated at the same time (on the same day). The run time of the cell below will be about 50 seconds.

In [ ]:
teams_left = teams.copy()

for i in range(len(results['date'])):
    current_date = results.iloc[i,0]
    #is team_1 eliminated?
    team1 = results.iloc[i,1]
    if (if_eliminated(team1, current_date)) and (team1 in teams_left):
        print(current_date)
        print(team1)
        teams_left.remove(team1)
    #is team_2 eliminated
    team2 = results.iloc[i,3]
    if (if_eliminated(team2, current_date)) and (team2 in teams_left):
        print(current_date)
        print(team2)
        teams_left.remove(team2)

The code below to produces a ranking of all teams at the end of the season. We display the teams by descending number of points, with the most number of points on the top and least number of points on the bottom.

In [ ]:
w, games_left = instance(datetime(2019, 10, 25), datetime(2020, 2, 29), datetime(2020, 2, 29))
w = w.sort_values(by=['w'], ascending=False)
w

**Q20:** Is the order of elimination the same as winning? In other words, does a team being eliminated first guarantee it's ranked the lowest at the end of the season?

**A:** <font color='blue'>No, they are not the same thing. Two teams can both be eliminated, but they still have more games left to play in the season. And the team eliminated first can end up winning more points in the remainder of the season. We just know that a team cannot win the whole season after being eliminated.</font>  

**Q21:** What does hockey elimination inform us about in terms of the team(s) that never get eliminated?

**A:** <font color='blue'>The winner must be among one of those teams. Those are the only teams with a chance of winning.</font>

In [ ]:
#also clean up women's data for people to just run code and see?

In [ ]:
#put it in .py file, and import to hide code
#if max flow is unique, there are necessary outcomes for what games need to happen